# Chapter 14: 모델 병합 및 내보내기

## 1. 학습 목표

* LoRA 어댑터를 베이스 모델에 병합하여 단일 모델로 만든다.
* 병합된 모델을 저장하고 GGUF 형식으로 변환한다.
* HuggingFace Hub에 모델을 업로드하고 모델 카드를 작성한다.

## 2. LoRA 병합 (Merge)

### 2.1 병합의 필요성

학습 시에는 메모리를 아끼기 위해 베이스 모델을 동결하고 LoRA 어댑터만 사용했지만, 실제 배포 시에는 어댑터를 따로 로드하는 연산 비용을 없애기 위해 두 모델을 합치는 것이 유리하다.

```text
학습 시: Base Model (동결) + LoRA Adapter (학습)
배포 시: Merged Model (단일 모델) → 추론 속도 향상, 관리 용이

```

### 2.2 병합 코드 구현

`peft` 라이브러리의 `merge_and_unload()` 메서드를 사용하면 간단히 병합할 수 있다. 이때 주의할 점은 병합을 위해 **베이스 모델을 FP16 또는 BF16으로 다시 로드**해야 한다는 것이다 (4-bit 상태로는 병합 불가).

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import os

def merge_lora_to_base(base_model_id, adapter_path, output_path):
    """
    LoRA 어댑터를 베이스 모델에 영구적으로 병합하여 저장하는 함수다.
    """
    print(f"1. 베이스 모델 로드 (FP16): {base_model_id}")

    # 병합을 위해 양자화 없이 FP16으로 로드한다.
    # (시스템 메모리가 부족하면 'cpu'로 로드 후 병합할 수도 있음)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )

    print(f"2. LoRA 어댑터 결합: {adapter_path}")
    model = PeftModel.from_pretrained(base_model, adapter_path)

    print("3. 가중치 병합 (Merge & Unload)...")
    merged_model = model.merge_and_unload()

    print(f"4. 병합된 모델 저장: {output_path}")
    merged_model.save_pretrained(output_path)

    # 토크나이저도 함께 저장해야 배포 시 편리하다.
    tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
    tokenizer.save_pretrained(output_path)

    print("병합 및 저장 완료!")
    return merged_model

# 실행 예시 (경로 수정 필요)
merge_lora_to_base("openai/gpt-oss-20b", "./GPT-OSS-20B-DPO-Final", "./GPT-OSS-20B-Merged")

1. 베이스 모델 로드 (FP16): openai/gpt-oss-20b


`torch_dtype` is deprecated! Use `dtype` instead!
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2. LoRA 어댑터 결합: ./GPT-OSS-20B-DPO-Final
3. 가중치 병합 (Merge & Unload)...
4. 병합된 모델 저장: ./GPT-OSS-20B-merged
병합 및 저장 완료!


GptOssForCausalLM(
  (model): GptOssModel(
    (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
    (layers): ModuleList(
      (0-23): 24 x GptOssDecoderLayer(
        (self_attn): GptOssAttention(
          (q_proj): Linear(in_features=2880, out_features=4096, bias=True)
          (k_proj): Linear(in_features=2880, out_features=512, bias=True)
          (v_proj): Linear(in_features=2880, out_features=512, bias=True)
          (o_proj): Linear(in_features=4096, out_features=2880, bias=True)
        )
        (mlp): GptOssMLP(
          (router): GptOssTopKRouter()
          (experts): GptOssExperts()
        )
        (input_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
        (post_attention_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
      )
    )
    (norm): GptOssRMSNorm((2880,), eps=1e-05)
    (rotary_emb): GptOssRotaryEmbedding()
  )
  (lm_head): Linear(in_features=2880, out_features=201088, bias=False)
)

## 3. GGUF 변환 (로컬 구동용)

병합된 모델을 개인 PC나 모바일 기기에서 돌리려면 `GGUF` 포맷으로 변환해야 한다. 이는 `llama.cpp` 프레임워크에서 사용하는 표준 포맷이다.

### 3.1 변환 프로세스

```bash
# 1. llama.cpp 도구 설치 (클론 및 빌드)
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
make

# 2. 필수 라이브러리 설치
pip install -r requirements.txt

# 3. 모델 변환 (HuggingFace 포맷 -> GGUF fp16)
# 사용법: python convert-hf-to-gguf.py [모델 경로] --outfile [출력 파일명]
python convert_hf_to_gguf.py ../gpt-oss-20b-merged --outfile gpt-oss-20b.gguf

# 4. 양자화 (선택: fp16 -> q4_k_m)
# 용량을 줄이기 위해 다시 4비트로 양자화한다.
./llama-quantize gpt-oss-20b.gguf gpt-oss-20b-q4_k_m.gguf q4_k_m

```

## 4. HuggingFace Hub 업로드

학습한 모델을 커뮤니티에 공유하거나 서버에 배포하기 위해 HuggingFace Hub에 업로드한다.

In [2]:
from huggingface_hub import HfApi, create_repo

def upload_to_hub(local_model_path, repo_id, token=None):
    """
    로컬에 저장된 모델 폴더를 HuggingFace Hub에 업로드한다.
    """
    api = HfApi(token=token)

    # 리포지토리 생성 (이미 있으면 건너뜀)
    try:
        create_repo(repo_id, private=True, exist_ok=True)
        print(f"리포지토리 준비 완료: {repo_id}")
    except Exception as e:
        print(f"리포지토리 확인 중 오류: {e}")

    # 폴더 전체 업로드
    print("업로드 시작 (시간이 소요될 수 있음)...")
    api.upload_folder(
        folder_path=local_model_path,
        repo_id=repo_id,
        repo_type="model"
    )
    print(f"업로드 완료: https://huggingface.co/{repo_id}")

# 실행 예시
upload_to_hub("./GPT-OSS-20B-Merged", "nowave/gpt-oss-20b-dpo-finetuned")

리포지토리 준비 완료: nowave/gpt-oss-20b-dpo-finetuned
업로드 시작 (시간이 소요될 수 있음)...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

업로드 완료: https://huggingface.co/nowave/gpt-oss-20b-dpo-finetuned


## 5. 모델 카드 작성 (Model Card)

모델의 신뢰도를 높이기 위해 `README.md` (모델 카드)를 작성해야 한다. 다음은 표준 템플릿이다.

```markdown
---
license: apache-2.0
language:
- ko
- en
tags:
- fine-tuned
- gpt-oss-20b
- lora
base_model: Qwen/Qwen3-14B
---

# GPT-OSS-20B Finetuned Model

## 모델 설명
이 모델은 `Qwen3-14B`를 기반으로 [금융/의료/일반] 도메인 데이터셋을 사용하여 Fine-tuning한 모델입니다.

## 학습 설정
- **방법**: QLoRA (4-bit)
- **Adapter Rank**: 16
- **Alpha**: 32
- **데이터셋**: [데이터셋 이름]
- **학습률**: 2e-4

## 사용법
```
```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "username/Qwen3-14B-finance-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
```

## 6. 요약

이 챕터에서는 학습 결과물을 배포 가능한 형태로 만드는 과정을 수행했다.

1.  **병합**: `merge_and_unload`를 통해 단일 모델로 변환하여 추론 효율을 높였다.
2.  **변환**: `llama.cpp`를 위한 GGUF 포맷 변환 방법을 확인했다.
3.  **배포**: HuggingFace Hub에 업로드하고 모델 카드를 작성하여 공유 준비를 마쳤다.

다음 챕터는 **Chapter 15: 모니터링 및 프로덕션**으로, 실제 서비스 운영 시 필요한 MLOps 요소들을 다룬다.